In [1]:
# Импорт всех необходимых библиотек
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVR, SVC
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer

# Загрузка данных
data = pd.read_excel('Данные_для_курсовои_Классическое_МО.xlsx')

#Уберем Unnamed т.к. далее он мешает
if 'Unnamed: 0' in data.columns:
    data = data.drop(columns=['Unnamed: 0'])

# Переместим названия колонок в отдельную переменную, пригодится
columns = data.columns.tolist()

# Быстрая проверка
data.head()

#'IC50, mM', 'CC50, mM', 'SI'

,"IC50, mM","CC50, mM",SI,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,6.239374,175.482382,28.125000,5.094096,5.094096,0.387225,0.387225,0.417362,42.928571,384.652,...,0,0,0,0,0,0,0,0,3,0
1,0.771831,5.402819,7.000000,3.961417,3.961417,0.533868,0.533868,0.462473,45.214286,388.684,...,0,0,0,0,0,0,0,0,3,0
2,223.808778,161.142320,0.720000,2.627117,2.627117,0.543231,0.543231,0.260923,42.187500,446.808,...,0,0,0,0,0,0,0,0,3,0
3,1.705624,107.855654,63.235294,5.097360,5.097360,0.390603,0.390603,0.377846,41.862069,398.679,...,0,0,0,0,0,0,0,0,4,0
4,107.131532,139.270991,1.300000,5.150510,5.150510,0.270476,0.270476,0.429038,36.514286,466.713,...,0,0,0,0,0,0,0,0,0,0


Подготовка данных для IC50
---

In [9]:
# Логарифмирование целевой переменной
data['log_IC50'] = np.log1p(data['IC50, mM'])

# Выделяем целевую переменную и признаки
X = data.drop(['IC50, mM', 'CC50, mM', 'SI'], axis=1)
y = data['IC50, mM']

# Разбиваем на train/test (чтобы честно оценивать модель)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Стандартизация признаков (важно для многих моделей регрессии)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Добавили data['log_IC50'] = np.log1p(data['IC50']) - логарифмирование улучшает регрессию
Выделили IC50 как целевую переменную. Провели разбиение для дальнейшего контроля качества моделей. Стандартизировали — важно для линейных и SVM моделей

Линейная регрессия
---

In [10]:
# Создаем импутер
imputer = SimpleImputer(strategy='median')
X_train_imputed = imputer.fit_transform(X_train_scaled)
X_test_imputed = imputer.transform(X_test_scaled)

# Линейная регрессия
linreg = LinearRegression()
linreg.fit(X_train_imputed, y_train)
y_pred_lin = linreg.predict(X_test_imputed)

# Оценка качества
print("Linear Regression:")
print("MAE:", mean_absolute_error(y_test, y_pred_lin))
print("MSE:", mean_squared_error(y_test, y_pred_lin))
print("R2:", r2_score(y_test, y_pred_lin))

Linear Regression:
MAE: 208.10151721932925
MSE: 146214.54510044036
R2: 0.5616534179776279


MAE: 208.1

    ошибка в среднем на 200 единиц IC50 (достаточно много)

MSE: 146214.5

    модель не справляется с нелинейной зависимостью в данных

R2: 0.5

    модель объясняет только 56% дисперсии в данных


линейная модель плохо справляется, потому что связь между признаками и IC50 нелинейная

Random Forest
---

In [14]:
# Случайный лес
rf = RandomForestRegressor(random_state=42)

# Подбор гиперпараметров через GridSearch
param_grid = {
    'n_estimators': [100],
    'max_depth': [5, 10],
    'min_samples_split': [2]
}

grid_rf = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_rf.fit(X_train, y_train)

# Лучшая модель
best_rf = grid_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test)

print("Random Forest (оптимальные параметры):", grid_rf.best_params_)
print("MAE:", mean_absolute_error(y_test, y_pred_rf))
print("MSE:", mean_squared_error(y_test, y_pred_rf))
print("R2:", r2_score(y_test, y_pred_rf))

Random Forest (оптимальные параметры): {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 100}
MAE: 34.494651725472075
MSE: 62507.8335820361
R2: 0.8126034917970261


R² = 0.812

    модель хорошо объясняет большую часть вариации IC50

MAE = 34.49

    модель ошибается больше в "длинных хвостах".

MSE = 62507.83

    Среднеквадратичная ошибка высокая из-за выбросов


Это говорит о том, что в данных есть сложные нелинейные зависимости, которые Random Forest отлично подхватывает.

Support Vector Regression (SVR)
---

In [16]:
# SVR работает лучше на стандартизированных данных
svr = SVR()

param_grid_svr = {
    'C': [1, 10, 100],
    'gamma': ['scale', 'auto'],
    'epsilon': [0.1, 0.5, 1]
}

grid_svr = GridSearchCV(svr, param_grid_svr, cv=5, scoring='neg_mean_squared_error')
grid_svr.fit(X_train_imputed, y_train)

best_svr = grid_svr.best_estimator_
y_pred_svr = best_svr.predict(X_test_imputed)

print("SVR (оптимальные параметры):", grid_svr.best_params_)
print("MAE:", mean_absolute_error(y_test, y_pred_svr))
print("MSE:", mean_squared_error(y_test, y_pred_svr))
print("R2:", r2_score(y_test, y_pred_svr))

SVR (оптимальные параметры): {'C': 100, 'epsilon': 1, 'gamma': 'scale'}
MAE: 211.1612855177411
MSE: 278467.01730231935
R2: 0.16516468893990488


MAE = 211.16


    средняя ошибка предсказания около 211 единиц по IC50.

MSE = 278467.017  

    видимо, из-за нескольких больших отклонений (так как MSE чувствительно к большим ошибкам).

R² = 0.165

    модель объясняет всего ~16% дисперсии данных
    
То есть SVR плохо справился в сравнении с другими моделями.

Сравнительная таблица
---

In [17]:
results = pd.DataFrame({
    'Модель': ['Linear Regression', 'Random Forest', 'SVR'],
    'MAE': [
        mean_absolute_error(y_test, y_pred_lin),
        mean_absolute_error(y_test, y_pred_rf),
        mean_absolute_error(y_test, y_pred_svr)
    ],
    'MSE': [
        mean_squared_error(y_test, y_pred_lin),
        mean_squared_error(y_test, y_pred_rf),
        mean_squared_error(y_test, y_pred_svr)
    ],
    'R2': [
        r2_score(y_test, y_pred_lin),
        r2_score(y_test, y_pred_rf),
        r2_score(y_test, y_pred_svr)
    ]
})

display(results)

,Модель,MAE,MSE,R2
0,Linear Regression,208.101517,146214.545100,0.561653
1,Random Forest,34.494652,62507.833582,0.812603
2,SVR,211.161286,278467.017302,0.165165



---
Linear Regression
--
Простая базовая модель.

MAE ≈ 208, что означает среднюю ошибку около 208 единиц IC50.

Объясняет около 56% дисперсии (R²=0.56).

Даёт стартовую точку, но сильно ограничена линейностью и не может уловить сложные зависимости в химических признаках.


---
Random Forest (лучший результат)
--
Существенно лучше: MAE снизилось до 34 — почти в 6 раз лучше базовой регрессии!

R² вырос до 0.81 — модель объясняет 81% дисперсии.

Отлично работает на табличных данных с нелинейностями, выбросами и сложными зависимостями.

---
3️SVR (Support Vector Regression)
--
На удивление слабый результат.

MAE ≈ 211 — даже хуже, чем у простой линейной регрессии.

SVR плохо работает в задачах с большим числом признаков, выбросами и без сложной настройки.

Подбор гиперпараметров через GridSearch не помог.

---
Основной вывод по задаче IC50:
--
Лучшей моделью является Random Forest.

Модель линейной регрессии можно оставить как базовый бенчмарк.

SVR не даёт ценного вклада и для данной задачи не рекомендован.

---
Почему Random Forest показывает себя лучше?
--

Автоматически учитывает важность признаков.

Работает с нелинейностями.

Более устойчив к выбросам (а они есть у IC50, как мы видели в EDA).